In [1]:
# from sklearn.model_selection import StratifiedShuffleSplit
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import json
from pycocotools.coco import COCO

In [2]:
data_type = 'eval' 

src_root = '/opt/ml/finalproject/data/' + data_type
src_file = src_root + '/resampled_errordel_'+ data_type +'.json'

coco_obj = COCO(src_file)

loading annotations into memory...
Done (t=0.79s)
creating index...
index created!


In [3]:
# Reading the json file
with open(src_file, 'r') as f:
    root = json.load(f)

In [4]:
img_id = coco_obj.getImgIds()
print(len(img_id), img_id[-1])

29759 29758


In [5]:
import numpy as np
li = [0]*38
X = [i for i in range(len(img_id))]
y = []
img_id = coco_obj.getImgIds()
for img in img_id:
    ann_id = coco_obj.getAnnIds(imgIds=img)
    one_hot = li.copy()
    for ann in ann_id:
        annotation = coco_obj.loadAnns(ann)
        cat_id = annotation[0]['category_id']
        one_hot = li.copy()
        one_hot[cat_id] = 1
    y.append(one_hot)
    
X = np.array(X)
y = np.array(y)

In [6]:
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [7]:
for train_index, test_index in mskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    break

TRAIN: [    0     1     2 ... 29754 29755 29756] TEST: [    5     6    11 ... 29753 29757 29758]


In [8]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

23807 5952
23807 5952


In [9]:
# root['images'] = []

In [10]:
train = []
for idx, image_id in enumerate(X_train):
    img_anno = coco_obj.loadImgs(int(image_id))
    img_anno[0]['id'] = idx
    train.append(img_anno[0])
    
test = []
for idx, image_id in enumerate(X_test):
    img_anno = coco_obj.loadImgs(int(image_id))
    img_anno[0]['id'] = idx
    test.append(img_anno[0])

In [11]:
train_json = root.copy()
test_json = root.copy()

train_json['images'] = []
train_json['images'] = train
test_json['images'] = []
test_json['images'] = test

In [12]:
with open("train.json", "w") as f:
    json.dump(train_json, f)
    
with open("test.json", "w") as f:
    json.dump(test_json, f)

In [13]:
# import plotly.express as px

# fig = px.parallel_categories()